In [ ]:
import pandas as pd
import numpy as np

# Load do dataset

In [ ]:
rs = 2

from lib.data import load_dataset
plain = True
if plain:
    path = 'dts/dts_plain/'
else:
    path = 'dts/dts_all/'
x_train, y_train, x_valid, y_valid = load_dataset(path + 'train.csv',rs=rs)
x_test = load_dataset('dts/dts_plain/test.csv',test_only=True,rs=rs)

## Normalização

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(x_train.values)

x_train = sc.transform(x_train.values)
x_valid = sc.transform(x_valid.values)
x_test = sc.transform(x_test.values)

# Feature Selection

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

selec_mod = SelectFromModel(estimator=LogisticRegression(random_state=rs, max_iter=2000)).fit(x_train, y_train)

x_train = selec_mod.transform(x_train)
x_valid = selec_mod.transform(x_valid)
x_test = selec_mod.transform(x_test)

# Modelo

In [ ]:
from my_nn import nnModel, dts
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from torch.utils.data import DataLoader
import torch

import numpy as np

class ensemble():
    def __init__(self,in_features = 1,rs = 2):
        self.rs = rs

        self.SVM = SVC(random_state = rs, tol=1e-2)
        self.random_forest = RandomForestClassifier(random_state=rs, n_estimators=20)
        self.nn = nnModel(inFeatures = in_features, random_state=rs)

    def fit(self,x,y):
        self.SVM.fit(x,y)
        self.random_forest.fit(x,y)

        train_dts = dts(x_train,y_train)
        valid_dts = dts(x_valid,y_valid)

        training_loader = DataLoader(train_dts, batch_size=32, shuffle=True)
        # valid_loader = DataLoader(valid_dts, batch_size=32, shuffle=True)

        losses, accs = self.nn.fit(
            trainloader=training_loader,
            learningRate=0.02,
            momentum=0.9,
            gamma=0.97,
            numEpochs=31,
            verbose=False
        )

    def predict(self,x):
        y_svm = self.SVM.predict(x)
        y_rf = self.random_forest.predict(x)
        y_nn = self.nn.predict(torch.tensor(x, dtype=torch.float32))

        comb = np.vstack((y_svm,y_rf,y_nn)).T.sum(axis=1)

        return 1*(comb >= 2)

        

# Treino

In [ ]:
from sklearn.metrics import accuracy_score

model = ensemble(in_features=x_train.shape[1],rs=rs)

model.fit(x_train,y_train)

In [ ]:
y_valid_pred = model.predict(x_valid)

accuracy = accuracy_score(y_true=y_valid, y_pred=y_valid_pred)

print(accuracy)

# Teste

In [ ]:
y_pred = model.predict(x_valid)

import pandas as pd
result = pd.DataFrame(data={
        "Id": range(len(y_pred)),
        "Category": y_pred.astype(int)
    }, index=None)

result.to_csv("result.csv", index=None)